In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
import os
import numpy as np
from operator import itemgetter 
from random import shuffle 

import matplotlib.pyplot as plt

In [ ]:
files = []
y_list = []

# load the binary files and add target variables
for class_n, c in enumerate(['kiss', 'handshake', 'hug', 'highfive']):
    f = os.listdir('output/'+c)
    f = list(map(lambda x: 'output/'+c+'/'+x, f))
    
    filtered = list(filter(lambda x: x.split('.')[-1] == 'npy', f))
    
    y_list.extend([class_n]*len(filtered))
    files.extend(filtered)

y = np.array(y_list)

# scale data to [0, 1] range
x = [np.load(x)/255 for x in files]

In [ ]:
print(x[0].dtype)
assert len(x) == len(y)